In [12]:
!pip install langchain openai wikipedia langchain-community numexpr "httpx==0.27.2"

In [ ]:
import os
from langchain.chat_models import ChatOpenAI
os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here" 

llm = ChatOpenAI(model="gpt-4o", temperature=0.1, max_tokens=2000)

In [17]:
# zeroshot ReAct agent with Wikipedia and LLM Math tools
from langchain.agents import initialize_agent, load_tools, AgentType
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

In [ ]:
# initialize the agent with the tools and LLM
# ZERO_SHOT_REACT_DESCRIPTION 
agent = initialize_agent(tools , llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, handle_parsing_errors=True, verbose=True) 
agent.invoke("요즘 여행가기 좋은 날은 몇월이야?")



> Entering new AgentExecutor chain...
여행하기 좋은 시기는 지역과 개인의 취향에 따라 다를 수 있습니다. 일반적으로 날씨가 온화하고 비가 적은 시기가 여행하기에 좋습니다. 예를 들어, 많은 사람들이 봄(4월~6월)과 가을(9월~11월)을 여행하기 좋은 시기로 꼽습니다. 이 시기에는 날씨가 쾌적하고 자연 경관도 아름답기 때문입니다. 하지만 특정 지역이나 활동에 따라 선호하는 시기가 다를 수 있으니, 여행 목적지의 기후와 개인의 취향을 고려하여 계획하는 것이 좋습니다. 

Final Answer: 일반적으로 여행하기 좋은 시기는 봄(4월~6월)과 가을(9월~11월)입니다.

> Finished chain.


{'input': '요즘 여행가기 좋은 날은 몇월이야?',
 'output': '일반적으로 여행하기 좋은 시기는 봄(4월~6월)과 가을(9월~11월)입니다.'}

In [20]:
# Conversational ReAct
from langchain.memory import ConversationBufferMemory

tools = load_tools(["llm-math"],  llm=llm)                      # llm-math 도구만 사용
memory = ConversationBufferMemory(memory_key="chat_history")    # 대화 기록 메모리 키 이름 설정
conversational_agent = initialize_agent(
    agent='conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,     # 디버깅 정보 출력
    max_iterations=3, # 에이전트 질문 해결 위한 최대 반복 횟수
    memory=memory,)
conversational_agent.invoke("Who is Geoffrey Hinton and how old is he? In addition, today is 2025-05-01.")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Geoffrey Hinton is a renowned computer scientist known for his work on artificial neural networks and deep learning. He has made significant contributions to the field of artificial intelligence and is often referred to as one of the "godfathers of AI." Hinton was born on December 6, 1947. Given that today is May 1, 2025, he would be 77 years old.

> Finished chain.


{'input': 'Who is Geoffrey Hinton and how old is he? In addition, today is 2025-05-01.',
 'chat_history': '',
 'output': 'Geoffrey Hinton is a renowned computer scientist known for his work on artificial neural networks and deep learning. He has made significant contributions to the field of artificial intelligence and is often referred to as one of the "godfathers of AI." Hinton was born on December 6, 1947. Given that today is May 1, 2025, he would be 77 years old.'}

In [21]:
# Self Ask With Search Agent
from langchain import hub
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent
from langchain_community.tools.tavily_search import TavilyAnswer

In [22]:
import os
os.environ["TAVILY_API_KEY"] = "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" 

In [23]:
tools = [TavilyAnswer(max_results=5, name="Intermediate Answer")]   # TavilyAnswer 도구 사용
prompt = hub.pull("hwchase17/self-ask-with-search")                 # "Self Ask With Search" 프롬프트를 가져옴
print(prompt.template) # 프롬프트 템플릿 출력


c:\Users\MAC\.conda\envs\venv_lmm\Lib\site-packages\langsmith\client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

In [29]:
agent = create_self_ask_with_search_agent(llm, tools, prompt) # "Self Ask With Search" 에이전트 생성

agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True, max_iterations=5, max_execution_time=10) # AgentExecutor 로 에이전트, 도구, 파싱 에러 처리 설정
agent_executor.invoke({"input": "Who lived longer, Muhammad Ali or Alan Turing?"})

{'input': 'Who lived longer, Muhammad Ali or Alan Turing?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [30]:
# ReAct document store
from langchain.docstore.wikipedia import Wikipedia        # Wikipedia 문서 저장소를 사용
from langchain.agents import Tool
from langchain.agents.react.base import DocstoreExplorer  # 문서 저장소 탐색기 생성

docstore=DocstoreExplorer(Wikipedia())

tools = [
    Tool(
        name="Search",
        func=docstore.search,  # 키워드 기반 검색. 
        description="docstore에서 용어 검색",
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup, # 정확한 용어 방식 검색.
        description="docstore에서 용어 검색",
    )
]

C:\Users\MAC\AppData\Local\Temp\ipykernel_10244\1283922457.py:6: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  docstore=DocstoreExplorer(Wikipedia())


In [33]:
react = initialize_agent(tools = tools,
                        llm = llm,
                        agent = AgentType.REACT_DOCSTORE, # 에이전트 유형 ReAct document store
                        handle_parsing_errors=True,       # 구문 오류 처리
                        max_iterations=1,                 # 에이전트 최대 반복 횟수
                        max_execution_time=5,             # 에이전트 작업 수행 최대 시간(초)
                        verbose = True,)

In [34]:
def query_data(query):
    try:
        response = react.invoke(query)  # 질의 전달 에이전트 실행
        print(f"response={response}")  
        return response                 # 검색 결과 반환
    except Exception as e:
        print(f"Error: {e}")            
        raise 

query = "샴 고양이에게 먹이주는 법은?"
response = query_data(query)
print(response['output']) 



> Entering new AgentExecutor chain...
Could not parse LLM Output: 샴 고양이에게 먹이를 주는 방법은 일반적인 고양이에게 먹이를 주는 방법과 크게 다르지 않습니다. 다음은 샴 고양이에게 적절한 먹이를 주기 위한 몇 가지 팁입니다:

1. **균형 잡힌 식단 제공**: 샴 고양이에게는 고양이의 나이, 체중, 활동 수준에 맞는 균형 잡힌 식단을 제공해야 합니다. 고양이 사료는 주로 건식 사료와 습식 사료로 나뉘며, 둘 다 적절히 섞어서 주는 것이 좋습니다.

2. **고품질 사료 선택**: 고양이의 건강을 위해 고품질의 사료를 선택하는 것이 중요합니다. 고양이 사료의 성분표를 확인하여 단백질 함량이 높은 제품을 선택하세요.

3. **정기적인 식사 시간**: 고양이에게 정기적인 식사 시간을 정해주는 것이 좋습니다. 하루에 두세 번 정해진 시간에 먹이를 주면 고양이가 규칙적인 식습관을 갖게 됩니다.

4. **적절한 양 조절**: 고양이의 체중과 건강 상태에 따라 먹이의 양을 조절해야 합니다. 과체중이 되지 않도록 주의하고, 필요에 따라 수의사와 상담하여 적절한 양을 결정하세요.

5. **신선한 물 제공**: 항상 신선한 물을 제공하여 고양이가 충분한 수분을 섭취할 수 있도록 합니다.

6. **간식은 적당히**: 간식은 고양이의 식단에 포함될 수 있지만, 너무 많이 주지 않도록 주의하세요. 간식은 전체 식단의 10%를 넘지 않도록 합니다.

샴 고양이는 활동적이고 사회적인 성격을 가지고 있으므로, 건강한 식단과 함께 충분한 운동과 놀이 시간을 제공하는 것도 중요합니다.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
Observation: Invalid or incomplete response
Thought:

> Finished chain.
response={'i